# 04 - Análise de Qualidade Consolidada dos Dados

## Objetivo

Este notebook tem como objetivo realizar uma **análise consolidada da qualidade dos dados**, considerando as tabelas finais utilizadas para análise e tomada de decisão no Data Warehouse (camada Gold).

A análise busca verificar a consistência, completude e coerência dos dados, avaliando se estes são adequados para responder às perguntas de negócio propostas no MVP.


In [0]:
# Leitura das tabelas da camada Gold
fact_maintenance = spark.table("fact_maintenance")
dim_machine = spark.table("dim_machine")

fact_maintenance.show(5)
dim_machine.show(5)


+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|machine_sk|udi|air_temperature_k|process_temperature_k|rotational_speed_rpm|torque_nm|tool_wear_min|machine_failure|twf|hdf|pwf|osf|rnf|
+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|      5086|  1|            298.1|                308.6|                1551|     42.8|            0|              0|  0|  0|  0|  0|  0|
|      8636|  2|            298.2|                308.7|                1408|     46.3|            3|              0|  0|  0|  0|  0|  0|
|      9622|  3|            298.1|                308.5|                1498|     49.4|            5|              0|  0|  0|  0|  0|  0|
|      4361|  4|            298.2|                308.6|                1433|     39.5|            7|              0|  0|  0|  0|  0|  0|
|      3017|  5|            298.2|

A visualização da tabela `fact_maintenance` confirma a consistência estrutural dos dados, com todas as métricas operacionais e indicadores de falha corretamente populados e sem evidência de valores ausentes nos registros exibidos.
A visualização da tabela `dim_machine` confirma a correta associação entre os identificadores de produto, o tipo de máquina e a chave substituta (`machine_sk`), assegurando integridade e rastreabilidade no modelo dimensional.
As visualizações iniciais não indicam anomalias evidentes, reforçando a adequação dos dados para as análises de qualidade subsequentes.


In [0]:
from pyspark.sql.functions import sum, when, col

fact_maintenance.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in fact_maintenance.columns
]).show()


+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|machine_sk|udi|air_temperature_k|process_temperature_k|rotational_speed_rpm|torque_nm|tool_wear_min|machine_failure|twf|hdf|pwf|osf|rnf|
+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+
|         0|  0|                0|                    0|                   0|        0|            0|              0|  0|  0|  0|  0|  0|
+----------+---+-----------------+---------------------+--------------------+---------+-------------+---------------+---+---+---+---+---+



In [0]:
dim_machine.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in dim_machine.columns
]).show()


+----------+----+----------+
|product_id|type|machine_sk|
+----------+----+----------+
|         0|   0|         0|
+----------+----+----------+



A contagem de valores nulos por coluna retornou zero para todos os atributos, confirmando a completude da tabela `fact_maintenance` e `dim_machine`.


In [0]:
%sql
SELECT
  MIN(air_temperature_k) AS min_air_temp,
  MAX(air_temperature_k) AS max_air_temp,
  MIN(process_temperature_k) AS min_process_temp,
  MAX(process_temperature_k) AS max_process_temp,
  MIN(rotational_speed_rpm) AS min_rpm,
  MAX(rotational_speed_rpm) AS max_rpm,
  MIN(torque_nm) AS min_torque,
  MAX(torque_nm) AS max_torque,
  MIN(tool_wear_min) AS min_tool_wear,
  MAX(tool_wear_min) AS max_tool_wear
FROM fact_maintenance;


min_air_temp,max_air_temp,min_process_temp,max_process_temp,min_rpm,max_rpm,min_torque,max_torque,min_tool_wear,max_tool_wear
295.3,304.5,305.7,313.8,1168,2886,3.8,76.6,0,253


Os valores mínimos e máximos das variáveis analisadas estão dentro de faixas esperadas, não indicando inconsistências ou anomalias nos dados.


In [0]:
%sql
SELECT COUNT(*) AS registros_orfaos
FROM fact_maintenance f
LEFT JOIN dim_machine d
  ON f.machine_sk = d.machine_sk
WHERE d.machine_sk IS NULL;


registros_orfaos
0


A verificação de integridade referencial indicou **zero registros órfãos**, confirmando que todas as chaves estrangeiras da tabela fato possuem correspondência válida na tabela de dimensão. Esse resultado assegura a consistência do modelo dimensional.


In [0]:
%sql
SELECT
  machine_failure,
  COUNT(*) AS total_registros
FROM fact_maintenance
GROUP BY machine_failure;


machine_failure,total_registros
0,9661
1,339


A distribuição dos registros mostra predominância de observações sem falha (**9.661 registros**), enquanto **339 registros** apresentam falha. Esse comportamento é consistente com cenários industriais reais, nos quais eventos de falha tendem a ser menos frequentes.

## Discussão da Qualidade dos Dados

A análise consolidada da qualidade dos dados confirmou alto nível de consistência e integridade nas tabelas da camada Gold. Não foram identificados valores nulos, inconsistências nos domínios das variáveis ou registros órfãos entre a tabela fato e a dimensão, validando a correção do modelo estrela. 

Dessa forma, os dados encontram-se aptos para suportar análises analíticas e responder às perguntas de negócio propostas neste MVP.


## Checklist de Qualidade

- [x] Análise de completude dos dados
- [x] Validação de domínios
- [x] Verificação de integridade referencial
- [x] Coerência dos indicadores de falha
- [x] Conjunto de dados apto para análise
